##Reading all CSV files

### October

In [0]:
df_oct = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/workspace/ecom/ecom_data/2019-Oct.csv")
)

### November

In [0]:
df_nov = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/workspace/ecom/ecom_data/2019-Nov.csv")
)


## Merging the Tables

In [0]:
df_all = df_oct.unionByName(df_nov)
df_all.show()

In [0]:
df_all.display()

## Writing as Managed Delta Table

In [0]:
df_all.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecom_orders")

### Confirming whether the Table Exists or Not

In [0]:
%sql
SELECT COUNT(*) FROM ecom_orders;

## Simulation

### Reusing Existing Data & Fragmenting It.
- Loop runs 3 times
- Each run writes 5 small files
- Total ≈ 15 new small files

In [0]:
for i in range(3):
    (
        df_nov
        .repartition(5)   ### Forcing Multiple Small Partitions
        .write
        .format("delta")
        .mode("append")
        .saveAsTable("ecom_orders")
    )

## Optimisation

In [0]:
%sql
OPTIMIZE ecom_orders;

### Checking File Count

In [0]:
%sql
DESCRIBE DETAIL ecom_orders;